# Preprocess c file

TCI percentiles per city-country

`Last update`: 2021-04-15

In [52]:
import subprocess
import pandas as pd
from loguru import logger 

def _create_cs_file(path_c):
    
    df = pd.read_csv(path_c)
    a = set(df.region_slug)
    
    df = df \
        .drop_duplicates('region_slug') 
    b = set(df.region_slug)
    
    df = df\
        .dropna()
    c = set(df.region_slug)

    print (a.difference(b))
    print (a.difference(c))
    
    for k in df.columns:
        if k != 'region_slug':
            df[k] = pd.to_numeric(df[k])
        if k == 'region_slug':
            df[k] = df[k].astype(str) 

    df_c = df \
        .assign(c_low_p01 = lambda x: (x['q25']-x['q01'])/(x['q75']-x['q25']),
                c_upp_p99 = lambda x: (x['q99']-x['q75'])/(x['q75']-x['q25']),
                c_low_p05 = lambda x: (x['q25']-x['q05'])/(x['q75']-x['q25']),
                c_upp_p95 = lambda x: (x['q95']-x['q75'])/(x['q75']-x['q25'])
               ) \
        [['region_slug', 'c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']]
    
    df_c['c_min'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].min(axis=1)
    df_c['c_max'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].max(axis=1)
    df_c['c_avg'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].mean(axis=1)
    df_c['c_lsa'] = df_c['c_max']*1.5
    df_c['c_lsb'] = df_c['c_max']*2
    df_c['c_lsc'] = 100*df_c['c_max']/df_c['c_max']
    
    return df_c

def _save_cs_file(): 
    
    logger.debug('writing file')
    
    df_c = _create_cs_file(path_c = '~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv')
    print(df_c.shape)
    print(df_c.describe())
    df_c.to_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv', index=False)
    

## Copy percentile

In [38]:
!cp ~/shared/spd-sdv-omitnik-waze/waze_tci/region_quantiles/quantiles/regions_quantiles-20210510.csv ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv

!sed "s/\[//g;s/\]//g;s/['\"]//g" ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [39]:
%%bash 

echo -e "region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95" | cat - ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

In [40]:
!rm ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [41]:
!head -2 ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95
aguascalientes,8039136.868743886,8862843413565.455,8887326406420.607,3829261.1993683483,5605996.229991711,6600013.058713919,6932325.881132038,7473441.530126513,8626943.901942156,9718372.086622398,11136396.36923769,11136396.36923769,2151065.466800198,14393234.680959623,6215087.576892198,10248917.983242217,3349729.7486610203,12964231.367048915


In [53]:
!wc -l ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

121 /home/soniame/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv


## Save c's file

In [145]:
_save_cs_file()

2021-05-20 22:15:14.979 | DEBUG    | __main__:_save_cs_file:46 - writing file


set()
{nan}
(109, 11)
        c_low_p01   c_upp_p99   c_low_p05   c_upp_p95       c_min       c_max  \
count  109.000000  109.000000  109.000000  109.000000  109.000000  109.000000   
mean     0.873449    1.521658    0.653693    0.778328    0.528733    1.596001   
std      0.421591    0.889621    0.342205    0.373704    0.216413    0.863302   
min      0.213163    0.506459    0.165875    0.308937    0.165875    0.506459   
25%      0.540299    0.972159    0.402509    0.529666    0.376233    1.052276   
50%      0.726099    1.280941    0.556643    0.675283    0.493846    1.451241   
75%      1.231027    1.763011    0.866212    0.922599    0.658330    1.789603   
max      2.017849    5.851183    1.580424    2.572249    1.482831    5.851183   

            c_avg       c_lsa       c_lsb         c_lsc  
count  109.000000  109.000000  109.000000  1.090000e+02  
mean     0.956782    2.394002    3.192003  1.000000e+02  
std      0.345273    1.294953    1.726604  5.296073e-15  
min      0.38869

In [146]:
print(len(df.region_slug.to_list()))
#df.region_slug.to_list()

110


In [125]:
#df[85:]

In [126]:
#df[df.region_slug == 'country_nicaragua']

### Add Brazil average

In [147]:
df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv')

In [129]:
#from src import utils
#conn = utils.connect_athena(path='../configs/athena.yaml')
#pd.read_sql_query("""
#select 
#    distinct country_iso_code, country_name,  region_slug
#from spd_sdv_waze_corona.prod_daily_daily_index
#where country_iso_code in ('BR', 'HN', 'UY') 
#    and region_slug NOT LIKE 'br_states%'
#""", conn)

../src/utils.py:158: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  athena_config = yaml.load(open(path, "r"))


In [148]:
from src import utils
conn = utils.connect_athena(path='../configs/athena.yaml')
br_regions = pd.read_sql_query("""
select 
    distinct region_slug
from spd_sdv_waze_corona.prod_daily_daily_index
where country_iso_code in ('BR')--, 'HN', 'UY') 
    and region_slug NOT LIKE 'br_states%'
""", conn)

../src/utils.py:158: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  athena_config = yaml.load(open(path, "r"))


In [149]:
df_br = df[df.region_slug.isin(br_regions.region_slug.to_list())]
df_br.shape

(22, 11)

In [150]:
import numpy as np
dicts_avg = {'region_slug':'country_brazil'}
keys = df_br.select_dtypes(include=np.number).columns.tolist()
for k in keys:
        dicts_avg[k] =df_br[k].mean()

In [151]:
df_ap = df.append(dicts_avg, ignore_index=True)
print(len(df_ap.region_slug.to_list()))
df_ap.to_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv', index=False)

110


### Regions missing

In [152]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
#from pathlib import Path
#home = str(Path.home())

import sys
sys.path.insert(0, '../')

from src import utils

In [153]:
conn = utils.connect_athena(path='../configs/athena.yaml')

In [154]:
qry = """
    select distinct region_slug
    from spd_sdv_waze_corona.prod_daily_daily_index
"""
dash_regions = pd.read_sql_query(qry, conn).sort_values('region_slug').region_slug.unique()

In [155]:
df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv')
cpar_regions = df.region_slug.to_list()

In [156]:
a = set(dash_regions)
b = set(cpar_regions)
len(a.intersection(b))

79

In [157]:
a.difference(b)

{'br_states_acre',
 'br_states_alagoas',
 'br_states_amapa',
 'br_states_amazonas',
 'br_states_bahia',
 'br_states_ceara',
 'br_states_distrito_federal',
 'br_states_espirito_santo',
 'br_states_goias',
 'br_states_maranhao',
 'br_states_mato_grosso',
 'br_states_mato_grosso_do_sul',
 'br_states_minas_gerais',
 'br_states_para',
 'br_states_paraiba',
 'br_states_parana',
 'br_states_pernambuco',
 'br_states_piaui',
 'br_states_rio_de_janeiro',
 'br_states_rio_grande_do_norte',
 'br_states_rio_grande_do_sul',
 'br_states_rondonia',
 'br_states_roraima',
 'br_states_santa_catarina',
 'br_states_sao_paulo',
 'br_states_sergipe',
 'br_states_tocantins',
 'country_honduras',
 'country_uruguay'}

In [158]:
b.difference(a)

{'asuncion',
 'barcelona',
 'barquisimeto',
 'belpoman',
 'bridgetown',
 'caracas',
 'ciudad_guayana',
 'cochabamba',
 'country_belize',
 'country_bolivia',
 'country_guyana',
 'country_haiti',
 'country_suriname',
 'country_the_bahamas',
 'country_venezuela',
 'florianopolis',
 'georgetown',
 'joinville',
 'juarez',
 'lapaz',
 'maracaibo',
 'maracay',
 'mendoza',
 'nassau',
 'paramaribo',
 'portauprince',
 'portofspain',
 'sanfernado',
 'sanpedrosula',
 'tijuana',
 'valencia'}